In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from utils.data_processing import *
from models.logistic_regression import LogisticRegression

In [2]:
articles_df = load_article_df()
articles_df.head()

Loaded 4604 articles in df of shape (4604, 1)


,article_name
0,Áedán_mac_Gabráin
1,Åland
2,Édouard_Manet
3,Éire
4,Óengus_I_of_the_Picts


In [3]:
links_df = load_links_df()
links_df.head()

Loaded 119882 links in df of shape (119882, 2)


,source,target
0,Áedán_mac_Gabráin,Bede
1,Áedán_mac_Gabráin,Columba
2,Áedán_mac_Gabráin,Dál_Riata
3,Áedán_mac_Gabráin,Great_Britain
4,Áedán_mac_Gabráin,Ireland


In [4]:
finished_df = load_finished_df()
finished_df.head()

Loaded 51318 finished paths in df of shape (51318, 7)


,hashIP,timestamp,duration,path,difficulty_rating,path_length,num_backward
0,6a3701d319fc3754,2011-02-15 03:26:49,166,"[14th_century, 15th_century, 16th_century, Pac...",NaN,9,0
1,3824310e536af032,2012-08-12 06:36:52,88,"[14th_century, Europe, Africa, Atlantic_slave_...",3.0,5,0
2,415612e93584d30e,2012-10-03 21:10:40,138,"[14th_century, Niger, Nigeria, British_Empire,...",NaN,8,0
3,64dd5cd342e3780c,2010-02-08 07:25:25,37,"[14th_century, Renaissance, Ancient_Greece, Gr...",NaN,4,0
4,015245d773376aab,2013-04-23 15:27:08,175,"[14th_century, Italy, Roman_Catholic_Church, H...",3.0,7,0


In [5]:
unifinished_df = load_unfinished_df()
unifinished_df.head()

Loaded 24875 unfinished paths in df of shape (24875, 8)


,hashIP,timestamp,duration,path,target_article,type_end,path_length,num_backward
0,2426091a53125110,2011-02-07 05:02:15,1804,[Obi-Wan_Kenobi],Microsoft,timeout,1,0
1,26141fd878806294,2011-02-07 05:14:11,1805,[Julius_Caesar],Caracas,timeout,1,0
2,2b015fb8181c48f2,2011-02-07 15:00:19,1818,"[Malawi, Democracy, Alexander_the_Great]",First_Crusade,timeout,3,0
3,53a53bc244e08a6a,2011-02-07 16:06:01,49,[Paraguay],Mount_St._Helens,restart,1,0
4,53a53bc244e08a6a,2011-02-07 17:18:25,1808,"[Paraguay, Bolivia]",Mount_St._Helens,timeout,2,0


In [6]:
all_games_df, _ = preprocess_and_concat_unfinished_and_finished(unifinished_df, finished_df)
all_games_df.head()

After filtering all paths after 2011-02-07 05:02:15
we kept 23245 paths out of 51318 finished paths
There are 24875 unfinished paths


,difficulty_rating,duration,finished,hashIP,num_backward,path,path_length,source,target,timestamp,type_end
0,NaN,166,True,6a3701d319fc3754,0,"[14th_century, 15th_century, 16th_century, Pac...",9,14th_century,African_slave_trade,2011-02-15 03:26:49,NaN
1,3.0,88,True,3824310e536af032,0,"[14th_century, Europe, Africa, Atlantic_slave_...",5,14th_century,African_slave_trade,2012-08-12 06:36:52,NaN
2,NaN,138,True,415612e93584d30e,0,"[14th_century, Niger, Nigeria, British_Empire,...",8,14th_century,African_slave_trade,2012-10-03 21:10:40,NaN
3,3.0,175,True,015245d773376aab,0,"[14th_century, Italy, Roman_Catholic_Church, H...",7,14th_century,John_F._Kennedy,2013-04-23 15:27:08,NaN
4,NaN,110,True,5295bca242be81fe,0,"[14th_century, Europe, North_America, United_S...",6,14th_century,John_F._Kennedy,2013-07-03 22:26:54,NaN


In [19]:
valid_games_df = prune_invalid_games(all_games_df, articles_df)
valid_games_df.head()

Pruning invalid games. Initially we have 48120 games
Pruned invalid games. Now we have 48092 valid games


,difficulty_rating,duration,finished,hashIP,num_backward,path,path_length,source,target,timestamp,type_end
0,NaN,166,True,6a3701d319fc3754,0,"[14th_century, 15th_century, 16th_century, Pac...",9,14th_century,African_slave_trade,2011-02-15 03:26:49,NaN
1,3.0,88,True,3824310e536af032,0,"[14th_century, Europe, Africa, Atlantic_slave_...",5,14th_century,African_slave_trade,2012-08-12 06:36:52,NaN
2,NaN,138,True,415612e93584d30e,0,"[14th_century, Niger, Nigeria, British_Empire,...",8,14th_century,African_slave_trade,2012-10-03 21:10:40,NaN
3,3.0,175,True,015245d773376aab,0,"[14th_century, Italy, Roman_Catholic_Church, H...",7,14th_century,John_F._Kennedy,2013-04-23 15:27:08,NaN
4,NaN,110,True,5295bca242be81fe,0,"[14th_century, Europe, North_America, United_S...",6,14th_century,John_F._Kennedy,2013-07-03 22:26:54,NaN


In [8]:
_ = dump_adjacency_list(links_df, articles_df)

UnicodeEncodeError: 'charmap' codec can't encode character '\xf8' in position 6: character maps to <undefined>

In [ ]:
_ = dump_unique_source_target_pairs(valid_games_df)

In [20]:
unique_game_stats_df = load_or_compute_unique_source_target_pair_stats()
unique_game_stats_df.head()

Loaded 29834 unique source-target pair stats


shortest_path_length  \
source       target                                         
14th_century African_slave_trade                        3   
             John_F._Kennedy                            3   
             Rainbow                                    3   
             Sodium                                     3   
             Elizabeth_I_of_England                     2   

                                     shortest_path_count  max_sp_pagerank  \
source       target                                                         
14th_century African_slave_trade                       3         0.002070   
             John_F._Kennedy                          18         0.009774   
             Rainbow                                   5         0.004548   
             Sodium                                    4         0.003013   
             Elizabeth_I_of_England                    1         0.000882   

                                     max_sp_avg_pagerank  avg_sp_avg_pagerank  \
source       target                                                             
14th_century African_slave_trade                0.000759             0.000562   
             John_F._Kennedy                    0.004316             0.002362   
             Rainbow                            0.001342             0.000626   
             Sodium                             0.001054             0.000713   
             Elizabeth_I_of_England             0.000508             0.000508   

                                     one_longer_path_count  max_ol_pagerank  \
source       target                                                           
14th_century African_slave_trade                        85         0.009774   
             John_F._Kennedy                          1758         0.009774   
             Rainbow                                   198         0.006847   
             Sodium                                    548         0.009774   
             Elizabeth_I_of_England                     61         0.006847   

                                     max_ol_avg_pagerank  avg_ol_avg_pagerank  \
source       target                                                             
14th_century African_slave_trade                0.003463             0.001167   
             John_F._Kennedy                    0.004792             0.002050   
             Rainbow                            0.002413             0.000922   
             Sodium                             0.003634             0.001034   
             Elizabeth_I_of_England             0.002172             0.000932   

                                     two_longer_pagerank  max_tl_pagerank  \
source       target                                                         
14th_century African_slave_trade                     300         0.006847   
             John_F._Kennedy                         300         0.009774   
             Rainbow                                 300         0.006847   
             Sodium                                  300         0.006847   
             Elizabeth_I_of_England                  300         0.006847   

                                     max_tl_avg_pagerank  
source       target                                       
14th_century African_slave_trade                0.002470  
             John_F._Kennedy                    0.004751  
             Rainbow                            0.002735  
             Sodium                             0.002960  
             Elizabeth_I_of_England             0.002537

In [21]:
node_stats_df = load_or_compute_node_stats()
node_stats_df.head()

Loaded 4604 node stats


,degree,closeness,betweenness,pagerank
article_name,,,,
10th_century,26,0.323931,5575.16,0.000684
11th_century,48,0.336125,6652.27,0.000595
12th_century,45,0.338172,14188.40,0.000769
13th_century,34,0.321593,8543.72,0.000810
14th_century,31,0.325205,5366.15,0.000642


In [22]:
valid_games_df = valid_games_df.merge(unique_game_stats_df, on=['source', 'target'], how='left')
valid_games_df.dropna(subset=['shortest_path_length'], inplace=True)
print("Number of valid games after shortest path length filtering: ", len(valid_games_df))

Number of valid games after shortest path length filtering:  48092


In [12]:
embeddings_df = load_embeddings()
embeddings_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/article_embeddings_smaller.csv'

In [ ]:
valid_games_df = compute_cosine_similarity(valid_games_df, embeddings_df)


In [23]:
valid_games_df = merge_with_node_data(valid_games_df, node_stats_df)

In [24]:
# Drop some variables before logistic regression
valid_games_df.drop(
    columns=[
        'hashIP', 'timestamp', 'path', 'difficulty_rating', 'type_end',
        'duration', 'path_length', 'num_backward'
    ], 
    inplace=True
)

valid_games_df = valid_games_df[valid_games_df['shortest_path_length'] > 0]

# The number of paths is better when log-transformed
valid_games_df['shortest_path_count'] = valid_games_df['shortest_path_count'].apply(lambda x: np.log(x + 1e-2))
valid_games_df['one_longer_path_count'] = valid_games_df['one_longer_path_count'].apply(lambda x: np.log(x + 1e-2))

# Change the type of finished to a float for logistic regression
valid_games_df['finished'] = valid_games_df['finished'].astype(float)

print(valid_games_df.shape)
print(valid_games_df.columns)

(48074, 23)
Index(['finished', 'source', 'target', 'shortest_path_length',
       'shortest_path_count', 'max_sp_pagerank', 'max_sp_avg_pagerank',
       'avg_sp_avg_pagerank', 'one_longer_path_count', 'max_ol_pagerank',
       'max_ol_avg_pagerank', 'avg_ol_avg_pagerank', 'two_longer_pagerank',
       'max_tl_pagerank', 'max_tl_avg_pagerank', 'degree_source',
       'closeness_source', 'betweenness_source', 'pagerank_source',
       'degree_target', 'closeness_target', 'betweenness_target',
       'pagerank_target'],
      dtype='object')


In [25]:
features_1 = ['shortest_path_length', 'shortest_path_count']

model_1 = LogisticRegression(valid_games_df, features_1)
model_1.fit()

Class distribution: finished
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64
Total number of samples: 46470
Optimization terminated successfully.
         Current function value: 0.649173
         Iterations 5
Training Set Metrics:
Threshold:   0.5030
F1 Score:    0.6181
Precision:   0.6182
Accuracy:    0.6182
              precision    recall  f1-score   support

         0.0     0.6277    0.6451    0.6363      4647
         1.0     0.6350    0.6174    0.6261      4647

    accuracy                         0.6313      9294
   macro avg     0.6314    0.6313    0.6312      9294
weighted avg     0.6314    0.6313    0.6312      9294



In [26]:
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               finished   No. Observations:                37176
Model:                          Logit   Df Residuals:                    37174
Method:                           MLE   Df Model:                            1
Date:                Mon, 09 Dec 2024   Pseudo R-squ.:                 0.06344
Time:                        21:03:03   Log-Likelihood:                -24134.
converged:                       True   LL-Null:                       -25768.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.7574      0.014    -52.498      0.000      -0.786      -0.729
x2             0.3402      0.013     26.288      0.000       0.315       0.366
==============================================================================
"""

In [27]:
features_2 = ['pagerank_target']

model_2 = LogisticRegression(valid_games_df, features_2, has_constant_term=True)
model_2.fit()

Class distribution: finished
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64
Total number of samples: 46470
Optimization terminated successfully.
         Current function value: 0.664087
         Iterations 6


C:\Users\bgabe\miniconda3\envs\ada\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bgabe\miniconda3\envs\ada\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\bgabe\miniconda3\envs\ada\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\b

Training Set Metrics:
Threshold:   0.4470
F1 Score:    0.6333
Precision:   0.6334
Accuracy:    0.6334
              precision    recall  f1-score   support

         0.0     0.6514    0.6383    0.6448      4647
         1.0     0.6454    0.6585    0.6519      4647

    accuracy                         0.6484      9294
   macro avg     0.6484    0.6484    0.6483      9294
weighted avg     0.6484    0.6484    0.6483      9294



In [28]:
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               finished   No. Observations:                37176
Model:                          Logit   Df Residuals:                    37174
Method:                           MLE   Df Model:                            1
Date:                Mon, 09 Dec 2024   Pseudo R-squ.:                 0.04192
Time:                        21:03:14   Log-Likelihood:                -24688.
converged:                       True   LL-Null:                       -25768.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0508      0.011      4.634      0.000       0.029       0.072
x1             0.7266      0.019     37.442      0.000       0.689       0.765
==============================================================================
"""

In [30]:
features_3 = [
    'shortest_path_length',
    'shortest_path_count', 'max_sp_pagerank', 'max_sp_avg_pagerank',
    'avg_sp_avg_pagerank', 'one_longer_path_count', 'max_ol_pagerank',
    'max_ol_avg_pagerank', 'avg_ol_avg_pagerank', 'two_longer_pagerank',
    'max_tl_pagerank', 'max_tl_avg_pagerank', #'cosine_similarity',
    'degree_source', 'closeness_source', 'betweenness_source',
    'pagerank_source', 'degree_target', 'closeness_target',
    'betweenness_target', 'pagerank_target'
]

model_3 = LogisticRegression(valid_games_df, features_3)
model_3.fit()

Class distribution: finished
0.0    0.5
1.0    0.5
Name: proportion, dtype: float64
Total number of samples: 46470
Optimization terminated successfully.
         Current function value: 0.635203
         Iterations 5
Training Set Metrics:
Threshold:   0.4750
F1 Score:    0.6328
Precision:   0.6365
Accuracy:    0.6343
              precision    recall  f1-score   support

         0.0     0.6630    0.5791    0.6182      4647
         1.0     0.6264    0.7056    0.6636      4647

    accuracy                         0.6423      9294
   macro avg     0.6447    0.6423    0.6409      9294
weighted avg     0.6447    0.6423    0.6409      9294



In [31]:
model_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               finished   No. Observations:                37176
Model:                          Logit   Df Residuals:                    37156
Method:                           MLE   Df Model:                           19
Date:                Mon, 09 Dec 2024   Pseudo R-squ.:                 0.08360
Time:                        21:03:48   Log-Likelihood:                -23614.
converged:                       True   LL-Null:                       -25768.
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.8162      0.027    -30.783      0.000      -0.868      -0.764
x2             0.0746      0.034      2.193      0.028       0.008       0.141
x3            -0.0679      0.039     -1.748      0.080      -0.144       0.008
x4             0.0412      0.049      0.847      0.397      -0.054       0.136
x5             0.0119      0.027      0.434      0.664      -0.042       0.065
x6             0.3057      0.039      7.785      0.000       0.229       0.383
x7            -0.0905      0.023     -4.014      0.000      -0.135      -0.046
x8             0.1438      0.036      3.973      0.000       0.073       0.215
x9            -0.0654      0.028     -2.314      0.021      -0.121      -0.010
x10            0.0098      0.015      0.641      0.522      -0.020       0.040
x11            0.0180      0.018      1.017      0.309      -0.017       0.053
x12            0.0449      0.023      1.912      0.056      -0.001       0.091
x13           -0.0420      0.024     -1.770      0.077      -0.088       0.005
x14           -0.2670      0.021    -12.683      0.000      -0.308      -0.226
x15            0.1175      0.025      4.719      0.000       0.069       0.166
x16           -0.0695      0.022     -3.114      0.002      -0.113      -0.026
x17            0.1831      0.025      7.249      0.000       0.134       0.233
x18           -0.1636      0.021     -7.723      0.000      -0.205      -0.122
x19           -0.1398      0.022     -6.219      0.000      -0.184      -0.096
x20            0.1912      0.028      6.862      0.000       0.137       0.246
==============================================================================
"""

In [32]:
combined_pred = valid_games_df.copy()[['finished']]

combined_pred['model_1_pred'] = model_1.predict(valid_games_df[features_1])
combined_pred['model_2_pred'] = model_2.predict(valid_games_df[features_2])
combined_pred['model_3_pred'] = model_3.predict(valid_games_df[features_3])

combined_pred['pred'] = combined_pred[['model_1_pred', 'model_2_pred', 'model_3_pred']].sum(axis=1)
combined_pred['pred'] = combined_pred['pred'].apply(lambda x: 1 if x >= 2 else 0)

print(classification_report(combined_pred['finished'], combined_pred['pred'], digits=4))

              precision    recall  f1-score   support

         0.0     0.6628    0.6014    0.6306     24839
         1.0     0.6123    0.6730    0.6412     23235

    accuracy                         0.6360     48074
   macro avg     0.6375    0.6372    0.6359     48074
weighted avg     0.6384    0.6360    0.6357     48074

